In [ ]:
import xarray as xr
import hvplot.xarray
import panel as pn
import pandas as pd
from holoviews.element import tiles
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import fsspec
import numpy as np
import glob
import dask
import param as pm
import geopandas as gpd
import re
from rasterio import features,transform
from dask.distributed import Client, LocalCluster
gv.extension('bokeh',logo=False)
hv.extension('bokeh',logo=False)
pn.extension()
cluster = LocalCluster(threads_per_worker=2)
client = Client(cluster)
client

In [ ]:
#Load Data
fs_map = fsspec.get_mapper('gs://ndvi_fuse/ndvi_fusion.zarr')
ndvi = xr.open_zarr(fs_map)
ndvi_wk_mean = ndvi.groupby('date.week').mean().sel(week = slice(13,48)).compute()
ndvi = ndvi.where((ndvi['date.dayofyear']<304)&(ndvi['date.dayofyear']>=91),drop=True).persist()
crnt_yr = int(ndvi['date.year'].max().values)
ndvi_yr_sums = ndvi.where((ndvi['date.dayofyear']>170)&(ndvi['date.dayofyear']<220)&(ndvi['date.year']!=crnt_yr),drop=True).groupby('date.year').sum().compute()
ndvi_dev = ndvi.sel(date=str(crnt_yr)).groupby('date.year').sum() - ndvi.where((ndvi['date.year']!=crnt_yr)&(ndvi['date.dayofyear']<pd.to_datetime(ndvi.date.max().values).dayofyear),drop=True).groupby('date.year').sum().mean(dim='year')
ndvi

In [ ]:
epsg=32613

In [ ]:
%%opts Scatter Violin {+axiswise}

sites = ['None','data/cper.geojson']
epsg=32613


def xr_2_affine(ds):
    x_size = len(ds.x)
    y_size = len(ds.y)
    x_scale = np.abs(np.diff(ds.x).mean())
    y_scale = np.abs(np.diff(ds.y).mean())
    upperleft_x = ds.x.min() - x_scale
    upperleft_y = ds.y.max() + y_scale
    return(transform.Affine(x_scale,0.0,upperleft_x,0,-y_scale,upperleft_y))

def shp2mask(shp,xr_object,fill=0, **kwargs):
    raster = features.rasterize(shp, fill=fill, transform=xr_2_affine(xr_object),
                                out_shape=xr_object.isel(date=0).ndvi.shape,dtype='int16', **kwargs)
    return xr.DataArray(raster,
                          coords=(xr_object.coords['y'].values,xr_object.coords['x']),
                          dims=('y', 'x'))
    


class Explorer(pm.Parameterized):
    trans = pm.Magnitude(default=1.0,label='NDVI Transperancy')
    sites = pm.ObjectSelector('data/cper.geojson', objects={s:s for s in sites})
    def __init__(self, **params):
        super(Explorer, self).__init__(**params)
        self.mask = None
        self.shp_info = None
        self.past_dict = None
        self.site_shp_outline = None
        self.crnt_yr = int(ndvi['date.year'].max().values)
    
    def ndvi_ts(self,index):
        if len(index)==0:
            
            ndvi_past = ndvi_yr_sums.where(self.mask!=0,drop=True).groupby('year').mean(...)
            min_year = ndvi_past.year.where(ndvi_past.min()==ndvi_past.ndvi,drop=True).year.values[0]
            max_year = ndvi_past.year.where(ndvi_past.max()==ndvi_past.ndvi,drop=True).year.values[0]
            yr_locs = np.where(np.isin(ndvi['date.year'].to_dataframe().year.values,[crnt_yr,min_year,max_year]))[0]
            ndvi_ts_past = ndvi.isel(date=yr_locs).where(self.mask!=0,drop=True).compute()
            print(crnt_yr)
            print(max_year)
            print(min_year)
            ndvi_plot = ndvi_ts_past.sel(date=str(max_year)).groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Best Yr: '+str(max_year)) * \
            ndvi_ts_past.sel(date=str(min_year)).groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Worst Yr: '+str(min_year)) * \
            ndvi_ts_past.sel(date=str(crnt_yr)).groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Current Yr: '+str(crnt_yr))
            ts_plot = ndvi_plot.opts(xlabel='Day Of Year',ylabel='NDVI',height=250,width=550).relabel('Entire Domain')
            
            ndvi_violin = xr.merge([ndvi_dev,self.mask.to_dataset(name='shp_mask')]).rename({'ndvi':'ndvi1'})
            v_plot = ndvi_violin.where(ndvi_violin.shp_mask!=0).hvplot.violin(y='ndvi1',by='shp_mask').opts(ylabel='NDVI Deviation',xlabel='Pasture',height=250,width=550)
            v_plot = v_plot.relabel('Cumulative NDVI Deviation: Entire Domain')
            p = ts_plot+v_plot
            return(p.opts(norm={'axiswise': False}))
        else:
            names = self.site_shp_outline.data.iloc[index,:].Pasture.tolist()
            
            ndvi_past = ndvi_yr_sums.where(self.mask.isin(index),drop=True).groupby('year').mean(...)
            min_year = ndvi_past.year.where(ndvi_past.min()==ndvi_past.ndvi,drop=True).year.values[0]
            max_year = ndvi_past.year.where(ndvi_past.max()==ndvi_past.ndvi,drop=True).year.values[0]
            yr_locs = np.where(np.isin(ndvi['date.year'].to_dataframe().year.values,[2019,min_year,max_year]))[0]
            ndvi_ts_past = ndvi.isel(date=yr_locs).where(self.mask.isin(index),drop=True).compute()

            ndvi_plot = ndvi_ts_past.sel(date=str(max_year)).groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Best Yr: '+str(max_year)) * \
            ndvi_ts_past.sel(date=str(min_year)).groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Worst Yr: '+str(min_year)) * \
            ndvi_ts_past.sel(date='2019').groupby('date.dayofyear').mean(...).hvplot.scatter('dayofyear','ndvi',label='Current Yr: '+str(self.crnt_yr))
            ts_plot = ndvi_plot.opts(xlabel='Day Of Year',ylabel='NDVI',height=250,width=550).relabel('Pasture NDVI: '+' + '.join(names))
            
            ndvi_violin = xr.merge([ndvi_dev,self.mask.to_dataset(name='shp_mask')]).rename({'ndvi':'ndvi1'})
            v_plot = ndvi_violin.where(ndvi_violin.shp_mask.isin(index)).hvplot.violin(y='ndvi1',by='shp_mask').opts(xlabel='Pasture',ylabel='NDVI Deviation',height=250,width=550)
            v_plot = v_plot.relabel('Cumulative NDVI Deviation: '+' + '.join(names))
            p = ts_plot+v_plot
            return(p.opts(norm={'axiswise': False}))

    @pm.depends('sites',watch=True)
    def veiwable(self,**kwargs):
        
        if self.sites != 'None':
            site_shp=gpd.read_file(self.sites).rename(columns={'Past_Name_':'Pasture'})
            self.shp_info = site_shp[['Pasture','geometry']].reset_index(drop=True).reset_index().rename(columns={'index':'id'})
            self.past_dict = {row.id:row.Pasture for _,row in self.shp_info.iterrows()}
            self.mask_shp = [(row.geometry,row.id+1) for _,row in self.shp_info.iterrows()]

            self.mask = shp2mask(self.mask_shp,ndvi)
            
            self.site_shp_outline = gv.Polygons(site_shp.to_crs(epsg),vdims='Pasture',crs=ccrs.epsg(epsg)).opts(tools=['hover','tap'], line_color='k',fill_alpha=0)
            sel = hv.streams.Selection1D(source=self.site_shp_outline)
            
#             sel.set_default(value=[2],param_name='index')
            ndv_ts_plot = hv.DynamicMap(self.ndvi_ts, kdims=[], streams=[sel])
            pp = pn.Column(ndvi.sel(date='2019').where(self.mask!=0).hvplot.image(x='x',
                                      y='y',
                                      crs=32613,
                                      cmap='viridis',
                                      min_width=500,
                                      min_height=450).opts(clim=(-.2,.6)).apply.opts(alpha=self.param.trans) * self.site_shp_outline * gv.tile_sources.EsriImagery())#,ndv_ts_plot)
            return(pp,ndv_ts_plot)
            #  
        else:
            return(gv.tile_sources.EsriImagery(width=1000))
        
        
        

explorer = Explorer(name='')
p1,p2 = explorer.veiwable()
llo = pn.Column(pn.Row(explorer.param,p1),p2)

llo.servable()